In [36]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

In [11]:
sq_eu = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/maps/QGIS/square_europe_updated.csv')
storm_dates = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_work.csv',
                          delimiter = ';')
name_storm = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/C3S_StormTracks_era5_19792021_0100_v3.csv',
                         delimiter=';')

landfall_eu = sq_eu.drop_duplicates(subset=['layer'], keep='first')
landfall_eu = landfall_eu.drop(columns=['lon_east', 'lon_west', 'lat_north', 'lat_south', 'path','center_lon','center_lat'])

name_storm_index = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]
name_storm = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]

In [3]:
'''import re

# Input string
line = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]

# Use regex to find the number after TRACK_ID and the text after 0
all_storms = len(name_storm)
for storm in range(0,all_storms):
    match = re.search(r'TRACK_ID (\d+).*?ENSMBLE_MEMB 0 (.+)', line['Time&Longitude&Latitude'].loc[storm])

if match:
    track_id_value = match.group(1)
    after_zero = match.group(2)

    print("TRACK_ID value:", track_id_value)
    print("Text after 0:", after_zero)'''

KeyError: 1

In [12]:
# in the column 'layer' rename each fid_reproj_tc_irad_i_interp by storm_i

landfall_eu = landfall_eu.rename(columns={'layer': 'storm_index'})
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('fid_reproj_tc_irad_', '')
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('_interp', '')

# clean the storm_dates dataframe

'''storm_eu_dates = storm_dates.drop(columns=['end_date', '3_hour_steps', '1_hour_steps'])
landfall_eu['storm_index'] = pd.to_numeric(landfall_eu['storm_index'], errors='coerce')

# keep only the name of the storms

name_storm = name_storm['Time&Longitude&Latitude'].str.split(' 0 ', n=1).str[1].str.strip()
name_storm = name_storm.apply(lambda x: x.replace('NAME', '', 1).strip() if x.startswith('NAME') else x)
name_storm.reset_index(drop=True, inplace=True)
name_storm = pd.DataFrame(name_storm)'''

"storm_eu_dates = storm_dates.drop(columns=['end_date', '3_hour_steps', '1_hour_steps'])\nlandfall_eu['storm_index'] = pd.to_numeric(landfall_eu['storm_index'], errors='coerce')\n\n# keep only the name of the storms\n\nname_storm = name_storm['Time&Longitude&Latitude'].str.split(' 0 ', n=1).str[1].str.strip()\nname_storm = name_storm.apply(lambda x: x.replace('NAME', '', 1).strip() if x.startswith('NAME') else x)\nname_storm.reset_index(drop=True, inplace=True)\nname_storm = pd.DataFrame(name_storm)"

In [4]:
'''test = storm_eu_dates['start_date'][0+1]
hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]
hours_to_add = int(hours_to_add['fid'].values[0])
date_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')
date_time_obj = date_time_obj + timedelta(hours=hours_to_add)

# convert back to string

date_time_obj.strftime('%Y-%m-%dT%H:%M:%S')'''

"test = storm_eu_dates['start_date'][0+1]\nhours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]\nhours_to_add = int(hours_to_add['fid'].values[0])\ndate_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')\ndate_time_obj = date_time_obj + timedelta(hours=hours_to_add)\n\n# convert back to string\n\ndate_time_obj.strftime('%Y-%m-%dT%H:%M:%S')"

In [5]:
storm_eu_arrivals = []
steps_added = []

for i in range(len(storm_eu_dates)):
    try:
        start = storm_eu_dates['start_date'][i]
        hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == i+1]
        hours_to_add = int(hours_to_add['fid'].values[0])
        start_reformat = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S')
        start_reformat = start_reformat + timedelta(hours=hours_to_add)
        storm_eu_arrivals.append(start_reformat.strftime('%Y-%m-%dT%H:%M:%S'))
        steps_added.append(hours_to_add)
    except:
        print(f'storm {i} not found')
        storm_eu_arrivals.append(-1)
        steps_added.append(-1)

storm 90 not found


In [6]:
# add the column 'arrival_date' to storm_dates

storm_dates['eu_landfall_date'] = storm_eu_arrivals
storm_dates['nb_steps_before_eu_landfall_1_hour'] = steps_added
#storm_dates['storm_name'] = name_storm

In [8]:
matched_data = []
df_storm_eu_arrivals= []
df_steps_added = []

df_storm_eu_arrivals = pd.DataFrame(storm_eu_arrivals)
df_steps_added = pd.DataFrame(steps_added)

# Loop through storm_dates and find matches
for idx in storm_eu_dates['storm_index']:
    if idx in name_storm.index:
        matched_storm_name = name_storm['Time&Longitude&Latitude'].loc[idx]
        #matched_storm_eu_arrivals = df_storm_eu_arrivals.loc[idx][0] they are alredy in the right order !
        #matched_steps_added = df_steps_added.loc[idx][0]
        matched_data.append([idx, matched_storm_name])

# Convert matched data to a new DataFrame
matched_df = pd.DataFrame(matched_data, columns=['storm_index_match','storm_name'])
print(matched_df)

    storm_index_match                   storm_name
0                   1                        DARIA
1                   2        C3S_STORM_TRACKS_ERA5
2                   3        C3S_STORM_TRACKS_ERA5
3                   7        C3S_STORM_TRACKS_ERA5
4                   4                       VIVIAN
..                ...                          ...
90                 91                       CIARA2
91                 92                       DENNIS
92                 93                         ALEX
93                 94  AIDEN C3S_STORM_TRACKS_ERA5
94                 95        C3S_STORM_TRACKS_ERA5

[95 rows x 2 columns]


In [24]:
storm_eu_arrivals

['1990-01-23T17:00:00',
 '1990-01-25T05:00:00',
 '1990-02-01T13:00:00',
 '1990-02-02T16:00:00',
 '1990-02-20T18:00:00',
 '1990-02-28T09:00:00',
 '1990-02-24T20:00:00',
 '1991-01-05T10:00:00',
 '1991-12-19T06:00:00',
 '1992-01-01T00:00:00',
 '1992-02-26T04:00:00',
 '1992-12-18T03:00:00',
 '1992-12-25T06:00:00',
 '1993-01-09T17:00:00',
 '1993-01-10T21:00:00',
 '1993-01-14T20:00:00',
 '1993-01-14T14:00:00',
 '1993-02-02T18:00:00',
 '1993-12-08T13:00:00',
 '1994-01-22T23:00:00',
 '1994-01-27T06:00:00',
 '1994-12-08T01:00:00',
 '1995-01-17T11:00:00',
 '1995-01-23T00:00:00',
 '1995-10-24T13:00:00',
 '1996-10-11T13:00:00',
 '1996-10-28T03:00:00',
 '1996-11-06T00:00:00',
 '1997-02-19T12:00:00',
 '1997-02-22T23:00:00',
 '1997-12-23T12:00:00',
 '1998-01-04T06:00:00',
 '1998-01-03T01:00:00',
 '1998-10-30T00:00:00',
 '1998-12-26T09:00:00',
 '1999-01-13T12:00:00',
 '1999-12-03T02:00:00',
 '1999-12-26T01:00:00',
 '1999-12-28T23:00:00',
 '2000-01-01T17:00:00',
 '2000-10-30T06:00:00',
 '2000-11-25T03:

In [9]:
final_df = pd.concat([storm_dates, matched_df], axis=1)
final_df = final_df.drop(columns=['Unnamed: 0.1'])

In [10]:
# save the new storm_dates

final_df.to_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_steps_index_landfall_corrected.csv')

In [2]:
# go back to scratch, dates are not correct

# original file for storm dates from C3S

original_storm_dates = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/C3S_StormTracks_era5_19792021_0100_v3.csv',
                                      delimiter=';')

In [17]:
# drop all columns except the 1st one 

transformed_storm_dates = original_storm_dates.drop(original_storm_dates.columns[1:], axis=1)

# then split the column 'Time&Longitude&Latitude' into 3 columns

transformed_storm_dates[['Time&Longitude&Latitude','Longitude','Latitude','whateverthatis']] = transformed_storm_dates['Time&Longitude&Latitude'].str.split(' ', n=3, expand=True)
transformed_storm_dates.rename(columns={'Time&Longitude&Latitude': 'Time'}, inplace=True)

# extract each line that starts with POINT_NUM (it represents the number of points in the track)
# Get the number of steps stored in 'Longitude' and 'Latitude' with the line that starts with 'POINT_NUM' in the column 'Time'
steps_longitude = transformed_storm_dates[transformed_storm_dates['Time'].str.startswith('POINT_NUM')]['Longitude'].to_frame().reset_index(drop=True)
steps_latitude = transformed_storm_dates[transformed_storm_dates['Time'].str.startswith('POINT_NUM')]['Latitude'].to_frame().reset_index(drop=True)
# Replace spaces (' ') with actual empty strings ('') in 'Longitude'
steps_longitude = steps_longitude.replace(' ', '')

# Use steps stored in Longitude where it's not an empty string, otherwise use the value in Latitude
steps = pd.DataFrame(np.where(steps_longitude != '', steps_longitude, steps_latitude), columns=['nb_steps'])

# extract each line that starts with TRACK_ID (it represents the index of the storm)

index = transformed_storm_dates[transformed_storm_dates['Time'].str.startswith('TRACK_ID')]['Longitude'].to_frame().reset_index(drop=True)
index.rename(columns={'Longitude': 'storm_index'}, inplace=True)
# substract 24 from the index to start at 1 and the storm names
index['storm_index'] = pd.to_numeric(index['storm_index']) - 24

# extract the names of the storms
index['name'] = transformed_storm_dates[transformed_storm_dates['Time'].str.startswith('TRACK_ID')]['whateverthatis'].to_frame().reset_index(drop=True)
index['name'] = index['name'].str.split(' 0 ', n=1).str[1].str.strip()
index['name'] = index['name'].apply(lambda x: x.replace('NAME', '', 1).strip() if x.startswith('NAME') else x)
# remove the C3S_STORM_TRACKS_ERA5 for the storm 95
index_storm_95 = index.index[index['storm_index'] == 95].values[0]
# Update the 'name' column in place
index.loc[index_storm_95, 'name'] = index.loc[index_storm_95, 'name'].replace('C3S_STORM_TRACKS_ERA5', '', 1)

# extract the date of start and end of the storm
# THIS PART IS NOT THE START DATE FOR WHATEVER REASON
'''start_date = transformed_storm_dates[transformed_storm_dates['Time'].str.startswith('TRACK_ID')]['whateverthatis'].to_frame().reset_index(drop=True)
start_date = start_date['whateverthatis'].str.split(' ', n=1).str[0].str.strip().to_frame()
start_date = start_date.rename(columns={'whateverthatis': 'start_date'})'''

# find the index of the row where 'Time' is "POINT_NUM"
start_indexes = transformed_storm_dates.index[transformed_storm_dates['Time'] == 'POINT_NUM'].to_list()
start_date = []
for start_index in start_indexes:
    previous_row_values = transformed_storm_dates.iloc[start_index + 1]['Time']
    start_date.append(previous_row_values)


# same for the end date
# find the index of the row where 'Time' is "TRACK_ID"
end_indexes = transformed_storm_dates.index[transformed_storm_dates['Time'] == 'TRACK_ID'].to_list()
# remove the first value in end_indexes and add the last index of the transformed_storm_dates
end_indexes.pop(0)

last_index = transformed_storm_dates.index[0]
end_indexes.append(last_index)
# Get the values of the row before "TRACK_ID"
end_date = []
for end_index in end_indexes:
    previous_row_values = transformed_storm_dates.iloc[end_index - 1]['Time']
    end_date.append(previous_row_values)

# transform the end date and start date into the format 'YYYY-MM-DDTHH:MM:SS'
end_date_formatted = pd.to_datetime(end_date,format="%Y%m%d%H")
end_date_formatted = end_date_formatted.strftime('%Y-%m-%dT%H:%M:%S')
end_date_formatted = pd.DataFrame(end_date_formatted, columns=['end_date'])

start_date_formatted = pd.to_datetime(start_date,format="%Y%m%d%H")
start_date_formatted = start_date_formatted.strftime('%Y-%m-%dT%H:%M:%S')
start_date_formatted = pd.DataFrame(start_date_formatted, columns=['start_date'])

In [18]:
# create one dataframe with index, name, start_date, end_date and nb_steps
final_storm_dates = pd.concat([index, start_date_formatted], axis=1)
final_storm_dates['end_date'] = end_date_formatted
final_storm_dates = pd.concat([final_storm_dates, steps], axis=1)

In [19]:
# double check if the nb_steps actually corresponds of steps of 3 hours between start and end date
test = pd.DataFrame(columns=['start_date'])

test['start_date'] = pd.to_datetime(final_storm_dates['start_date'])
test['end_date'] = pd.to_datetime(final_storm_dates['end_date'])
test['diff'] = test['end_date'] - test['start_date']
test['diff'] = test['diff'] / np.timedelta64(1, 'h')
test['diff'] = test['diff'] / 3
test['diff'] = test['diff'].astype(int)+1
test['nb_steps'] = final_storm_dates['nb_steps'].astype(int)

if test['diff'].equals(test['nb_steps']):
    print('The number of steps is correct')
else:
    print('The number of steps is not correct at storm number: ')
    print(test[test['diff'] != test['nb_steps']])

The number of steps is correct


In [91]:
# creation of the square with a certain degree of latitude and longitude

number_of_storms = len(final_storm_dates)
degree = 4
path_tracks = '/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/ALL_TRACKS/'

for storm in range(0,number_of_storms):

    lon_east = []
    lon_west = []
    lat_north = []
    lat_south = []

    for step in range(start_indexes[storm], end_indexes[storm]-1):
        # get the latitude and longitude of the storm
        lat = float(transformed_storm_dates['Latitude'][step+1])
        lon = float(transformed_storm_dates['Longitude'][step+1])

        lon_east_temp = (lon + degree) % 360
        lon_west_temp = (lon - degree + 360 ) % 360
        lat_north_temp = (lat + degree)
        lat_south_temp = (lat - degree)

        lon_east.append(round(lon_east_temp, 6))
        lon_west.append(round(lon_west_temp, 6))
        lat_north.append(round(lat_north_temp, 6))
        lat_south.append(round(lat_south_temp, 6))

    # create the square
    storm_str = str(storm+1)
    square_name = f'storm_{storm_str}'
    square = pd.DataFrame({'lon_east': lon_east, 'lon_west': lon_west, 'lat_north': lat_north, 'lat_south': lat_south})

    # Define the folder and file path
    folder = 'tracks_3h/'
    file_name = f'{square_name}.csv'  # Specify the file name
    full_path = os.path.join(path_tracks, folder)

    # Create the folder if it doesn't exist
    os.makedirs(full_path, exist_ok=True)

    # Define the full path to the file, including the file name
    file_path = os.path.join(full_path, file_name)

    # Save the square in a CSV file in the folder 'tracks_3h'
    # If the file doesn't exist, it will be created
    square.to_csv(file_path, index=False)

In [92]:
# now the same but the conversion from 0-360 to -180-180

for storm in range(0,number_of_storms):
    
        lon_east = []
        lon_west = []
        lat_north = []
        lat_south = []
    
        for step in range(start_indexes[storm], end_indexes[storm]-1):
            # get the latitude and longitude of the storm
            lat = float(transformed_storm_dates['Latitude'][step+1])
            lon = float(transformed_storm_dates['Longitude'][step+1])
    
            lon_east_temp = (lon + degree) % 360
            lon_west_temp = (lon - degree + 360 ) % 360
            lat_north_temp = (lat + degree)
            lat_south_temp = (lat - degree)
    
            if lon_east_temp > 180:
                lon_east_temp = lon_east_temp - 360
            if lon_west_temp > 180:
                lon_west_temp = lon_west_temp - 360
    
            lon_east.append(round(lon_east_temp, 6))
            lon_west.append(round(lon_west_temp, 6))
            lat_north.append(round(lat_north_temp, 6))
            lat_south.append(round(lat_south_temp, 6))
    
        # create the square
        storm_str = str(storm+1)
        square_name = f'storm_{storm_str}'
        square = pd.DataFrame({'lon_east': lon_east, 'lon_west': lon_west, 'lat_north': lat_north, 'lat_south': lat_south})
    
        # Define the folder and file path
        folder = 'tracks_3h_GIS_friendly/'
        file_name = f'{square_name}.csv'  # Specify the file name
        full_path = os.path.join(path_tracks, folder)
    
        # Create the folder if it doesn't exist
        os.makedirs(full_path, exist_ok=True)
    
        # Define the full path to the file, including the file name
        file_path = os.path.join(full_path, file_name)
    
        # Save the square in a CSV file in the folder 'tracks_3h'
        # If the file doesn't exist, it will be created
        square.to_csv(file_path, index=False)

In [93]:
# now extrapolate the steps from 3 hours to 1 hour

def interpolate_vector(data, factor):
    n = len(data)
    # X interpolation points. For factor=4, it is [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, ...]
    x = np.linspace(0, n - 1, (n - 1) * factor + 1)
    # Alternatively:
    # x = np.arange((n - 1) * factor + 1) / factor
    # X data points: [0, 1, 2, ...]
    xp = np.arange(n)
    # Interpolate
    return np.round(np.interp(x, xp, np.asarray(data)),6)
